# sPlot with simple Gaussian and polynomial PDFs

Fit a pseudo data missing mass distribution to produce sWeights then use them to plot Signal weighted distribution for other variables.

Note the weights are stored in a Weights object which then can be loaded into other fits. [Weights.h](https://github.com/dglazier/brufit/blob/master/core/Weights.h)

Load brufit using ROOT python bindings and initialise jsroot for drawing histograms.

In [ ]:
import ROOT
ROOT.gROOT.ProcessLine(".x $BRUFIT/macros/LoadBru.C")
%jsroot

First you will need to generate some data. This is done with a ROOT macro generating random numbers from TF1 functions. It can be executed from the following notebook GenerateData
Link to [Generate Data](GenerateData.ipynb)

Create the sPlot fit manager and set the ouput directory for fit results, plots and weights

In [ ]:
splot = ROOT.sPlot()
splot.SetUp().SetOutDir("out")

Define the fit variable as Mmiss which is the name of a branch in the tree and set the fit range to 0-10

In [ ]:
splot.SetUp().LoadVariable("Mmiss[0,10]");

Set the name of the event ID variable. The input tree should have a double branch with a unique event ID number, in this case it is fgID.

In [ ]:
splot.SetUp().SetIDBranchName("fgID")

Make a signal PDF. Here we will use a Gaussian distribution with mean and width parameters smean (initial value 6, allowed values between 4-7) and swidth (initial value 0.2, allowed values between 0.0001 and 3). The PDF is given the name Signal.

We then load it into the total fit PDF with LoadSpeciesPDF

In [ ]:
splot.SetUp().FactoryPDF("Gaussian::Signal( Mmiss, smean[6,4,7], swidth[0.2,0.0001,3] )");
splot.SetUp().LoadSpeciesPDF("Signal")

Make a background PDF as a 2nd degree Chebychev polynomial with coefficients a0 (starting -0.1 between -1 and 1) and a1. The PDF is given the name BG

In [ ]:
splot.SetUp().FactoryPDF("Chebychev::BG(Mmiss,{a0[-0.1,-1,1],a1[0.1,-1,1]})");
splot.SetUp().LoadSpeciesPDF("BG",1)

Load the data giving the tree name then the file name.

In [ ]:
dirname=ROOT.gSystem.pwd();
splot.LoadData("MyModel",dirname+"/Data.root")

Run the fit locally, this will also create the sWeights for each event. When it is finished it will dosplay a plot of the signal and background fit to Mmiss and also show the residual and pull plots between the fit and data

In [ ]:
ROOT.Here.Go(splot)

Now we will draw the resulting weighted distributions. First create canvases for drawing on.

In [ ]:
from ROOT import TCanvas
canvas = TCanvas("WeightedPlots","WeightedPlots")
canvas.Divide(2,2);

Now get the truth tree so we can compare true to weighted distributions.

In [ ]:
fileTree =  ROOT.FiledTree.Read("MyModel","Data.root")
trueTree = fileTree.Tree()

And draw the weighted distributions using sPlot::DrawWeighted. The first string takes standard ROOT TTree::Draw arguments. The second is the name of the PDF corresponding to the species you want to draw.

trueTree is just a normal ROOT TTree.

The plots will appear when canvas.Draw() is called, which is in the following cell after the background histograms have been created. The weighted distributions will be points with error bars, the true distributions will be solid line histograms.


In [ ]:
canvas.cd(1)
splot.DrawWeighted("M1>>hM1(100,0,10)","Signal");
trueTree.Draw("M1","Sig==1","same hist");
canvas.cd(2)
splot.DrawWeighted("M2>>hM2(100,0,10)","Signal");
trueTree.Draw("M2","Sig==1","same hist");

Now for our background weighted distributions.

In [ ]:
canvas.cd(3)
splot.DrawWeighted("M1>>hM1_BG(100,0,10)","BG");
trueTree.Draw("M1","Sig==-1","same hist");
canvas.cd(4)
splot.DrawWeighted("M2>>hM2_BG(100,0,10)","BG");
trueTree.Draw("M2","Sig==-1","same hist");
canvas.Draw()

If we want to save the weighted tree with all branches inclduing weights you need to use the following line

In [ ]:
splot.DeleteWeightedTree()